In [1]:
"""
Start at root directory or anywhere
"""

import os
import datetime
from dotenv import load_dotenv
from lightmem.memory.lightmem import LightMemory

LOGS_ROOT = "./logs"
RUN_TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_LOG_DIR = os.path.join(LOGS_ROOT, RUN_TIMESTAMP)
os.makedirs(RUN_LOG_DIR, exist_ok=True)

os.environ["HF_HOME"] = "/home/mila/b/baldelld/scratch/"
os.environ["OPENROUTER_API_BASE"] = "https://openrouter.ai/api/v1"

load_dotenv()
# CRITICAL: Remove OPENROUTER_API_KEY to avoid the buggy branch
api_key = os.getenv("OPENROUTER_API_KEY")
if "OPENROUTER_API_KEY" in os.environ:
    del os.environ["OPENROUTER_API_KEY"]

# Now set OPENAI keys instead
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

# Rest of your code...
os.environ["HF_HOME"] = "/home/mila/b/baldelld/scratch/"

API_KEY = api_key  # Use the saved key
API_BASE_URL = 'https://openrouter.ai/api/v1'
LLM_MODEL = 'openai/gpt-oss-20b'
EMBEDDING_MODEL_PATH='sentence-transformers/all-MiniLM-L6-v2'
LLMLINGUA_MODEL_PATH='microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank'

config_dict = {
    "pre_compress": True,
    "pre_compressor": {
        "model_name": "llmlingua-2",
        "configs": {
            "llmlingua_config": {
                "model_name": LLMLINGUA_MODEL_PATH,
                "device_map": "cuda",
                "use_llmlingua2": True,
            },
        }
    },
    "topic_segment": True,
    "precomp_topic_shared": True,
    "topic_segmenter": {
        "model_name": "llmlingua-2",
    },
    "messages_use": "user_only",
    "metadata_generate": True,
    "text_summary": True,
    "memory_manager": {
        "model_name": 'openai', # such as 'openai' or 'ollama' ...
        "configs": {
            "model": LLM_MODEL,
            "api_key": API_KEY,
            "max_tokens": 16000,
            # "openrouter_base_url": API_BASE_URL # API model specific, such as 'openai_base_url' or 'deepseek_base_url' ...
        }
    },
    "extract_threshold": 0.1,
    "index_strategy": "embedding",
    "text_embedder": {
        "model_name": "huggingface",
        "configs": {
            "model": EMBEDDING_MODEL_PATH,
            "embedding_dims": 384,
            "model_kwargs": {"device": "cuda"},
        },
    },
    "retrieve_strategy": "embedding",
    "embedding_retriever": {
        "model_name": "qdrant",
        "configs": {
            "collection_name": "my_long_term_chat",
            "embedding_model_dims": 384,
            "path": "./my_long_term_chat", 
        }
    },
    "update": "offline",
    "logging": {
        "level": "DEBUG",
        "file_enabled": True,
        "log_dir": RUN_LOG_DIR,
    }
}

lightmem = LightMemory.from_config(config_dict)

pre_compressor:llmlingua-2
pre_compressor:llmlingua_config={'model_name': 'microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank', 'device_map': 'cuda', 'use_llmlingua2': True} llmlingua2_config={'max_batch_size': 50, 'max_force_token': 100} compress_config={'instruction': '', 'rate': 0.8, 'target_token': -1}


2025-11-17 11:27:14 - LightMemory - INFO - Initializing LightMemory with provided configuration
2025-11-17 11:27:14 - LightMemory - INFO - Initializing pre-compressor
`torch_dtype` is deprecated! Use `dtype` instead!
2025-11-17 11:27:36 - LightMemory - INFO - Initializing topic segmenter
2025-11-17 11:27:36 - LightMemory - INFO - Initializing memory manager
2025-11-17 11:27:37 - LightMemory - INFO - Initializing text embedder
2025-11-17 11:27:38 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-11-17 11:27:39 - LightMemory - INFO - Initializing embedding retriever
2025-11-17 11:27:39 - LightMemory - INFO - LightMemory initialization completed successfully


In [11]:

### Add Memory
session = {
"timestamp": "2025-01-10",
"turns": [
    [
        {"role": "user", "content": "My favorite ice cream flavor is pistachio, and my dog's name is Rex."}, 
        {"role": "assistant", "content": "Got it. Pistachio is a great choice."}], 
    ]
}


for turn_messages in session["turns"]:
    timestamp = session["timestamp"]
    for msg in turn_messages:
        msg["time_stamp"] = timestamp
        
    store_result = lightmem.add_memory(
        messages=turn_messages,
        force_segment=True,
        force_extract=True
    )

2025-11-17 10:29:29 - LightMemory - INFO - ========== START add_memory_20251117_102929_137443 ==========
2025-11-17 10:29:29 - LightMemory - INFO - force_segment=True, force_extract=True
2025-11-17 10:29:29 - LightMemory - INFO - [add_memory_20251117_102929_137443] Target compression rate: 0.8
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
2025-11-17 10:29:29 - LightMemory - INFO - [add_memory_20251117_102929_137443] Generated 1 segments
2025-11-17 10:29:29 - LightMemory - INFO - [add_memory_20251117_102929_137443] Extraction triggered 1 times, extract_list length: 1
2025-11-17 10:29:29 - LightMemory 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 10:29:32 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection my_long_term_chat


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 10:29:32 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection my_long_term_chat
2025-11-17 10:29:32 - LightMemory - INFO - [offline_update_20251117_102932_437376] Successfully inserted 2 entries to vector database


In [12]:
lightmem.construct_update_queue_all_entries()
lightmem.offline_update_all_entries(score_threshold=0.8)

2025-11-17 10:29:51 - LightMemory - INFO - ========== START construct_queue_20251117_102951_861595 ==========
2025-11-17 10:29:51 - LightMemory - INFO - [construct_queue_20251117_102951_861595] Parameters: top_k=20, keep_top_n=10, max_workers=8
2025-11-17 10:29:51 - LightMemory - INFO - [construct_queue_20251117_102951_861595] Retrieved 2 entries from vector database
2025-11-17 10:29:51 - LightMemory - INFO - [construct_queue_20251117_102951_861595] Starting parallel queue construction with 8 workers
2025-11-17 10:29:51 - LightMemory - INFO - [construct_queue_20251117_102951_861595] Queue construction completed: 2 updated, 0 skipped, nonempty_queues=2, empty_queues=0
2025-11-17 10:29:51 - LightMemory - INFO - ========== END construct_queue_20251117_102951_861595 ==========
2025-11-17 10:29:51 - LightMemory - INFO - ========== START offline_update_all_20251117_102951_929253 ==========
2025-11-17 10:29:51 - LightMemory - INFO - [offline_update_all_20251117_102951_929253] Parameters: scor

In [14]:
question = "What is the name of my dog?"
related_memories = lightmem.retrieve(question, limit=5)
print(related_memories)

2025-11-17 11:06:51 - LightMemory - INFO - ========== START retrieve_20251117_110651_203111 ==========
2025-11-17 11:06:51 - LightMemory - INFO - [retrieve_20251117_110651_203111] Query: What is the name of my dog?
2025-11-17 11:06:51 - LightMemory - INFO - [retrieve_20251117_110651_203111] Parameters: limit=5, filters=None


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 11:06:51 - LightMemory - INFO - [retrieve_20251117_110651_203111] Searching vector database
2025-11-17 11:06:51 - LightMemory - INFO - [retrieve_20251117_110651_203111] Found 2 results
2025-11-17 11:06:51 - LightMemory - INFO - [retrieve_20251117_110651_203111] Formatted 2 results into output string
2025-11-17 11:06:51 - LightMemory - INFO - ========== END retrieve_20251117_110651_203111 ==========


2025-01-10T00:00:00.000 Fri User's dog is named Rex.
2025-01-10T00:00:00.000 Fri User's favorite ice cream flavor is pistachio.


## Test Forking Logic (SCT-style branching)

We'll simulate a conversation that forks:
1. **Main conversation**: Add 3 turns about travel preferences
2. **Fork point**: Clone memories to a branch
3. **Branch A**: Continue with "Paris" hypothesis
4. **Branch B**: Continue with "Tokyo" hypothesis
5. **Verify**: Each branch has pre-fork memories + its own post-fork memories


2025-11-17 11:27:39 - LightMemory - INFO - Initializing LightMemory with provided configuration
2025-11-17 11:27:39 - LightMemory - INFO - Initializing pre-compressor


pre_compressor:llmlingua-2
pre_compressor:llmlingua_config={'model_name': 'microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank', 'device_map': 'cuda', 'use_llmlingua2': True} llmlingua2_config={'max_batch_size': 50, 'max_force_token': 100} compress_config={'instruction': '', 'rate': 0.8, 'target_token': -1}


2025-11-17 11:27:39 - LightMemory - INFO - Initializing topic segmenter
2025-11-17 11:27:39 - LightMemory - INFO - Initializing memory manager
2025-11-17 11:27:39 - LightMemory - INFO - Initializing text embedder
2025-11-17 11:27:39 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-11-17 11:27:40 - LightMemory - INFO - Initializing embedding retriever
2025-11-17 11:27:40 - LightMemory - INFO - LightMemory initialization completed successfully
2025-11-17 11:27:40 - LightMemory - INFO - ========== START add_memory_20251117_112740_201190 ==========
2025-11-17 11:27:40 - LightMemory - INFO - force_segment=True, force_extract=True


MAIN CONVERSATION: Pre-fork turns


2025-11-17 11:27:40 - LightMemory - INFO - [add_memory_20251117_112740_201190] Target compression rate: 0.8
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
2025-11-17 11:27:40 - LightMemory - INFO - [add_memory_20251117_112740_201190] Generated 1 segments
2025-11-17 11:27:40 - LightMemory - INFO - [add_memory_20251117_112740_201190] Extraction triggered 1 times, extract_list length: 1
2025-11-17 11:27:40 - LightMemory - INFO - [add_memory_20251117_112740_201190] Assigned timestamps to 1 items
2025-11-17 11:27:40 - LightMemory - INFO - [add_memory_20251117_112740_201190] Starting metadata generation
202

...

2025-11-17 11:27:46 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection fork_test_main
2025-11-17 11:27:46 - LightMemory - INFO - [offline_update_20251117_112746_625234] Successfully inserted 1 entries to vector database
2025-11-17 11:27:46 - LightMemory - INFO - ========== START retrieve_20251117_112746_642148 ==========
2025-11-17 11:27:46 - LightMemory - INFO - [retrieve_20251117_112746_642148] Query: What are my travel preferences?
2025-11-17 11:27:46 - LightMemory - INFO - [retrieve_20251117_112746_642148] Parameters: limit=5, filters=None


✓ Added turn 3

RETRIEVAL at fork point:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 11:27:46 - LightMemory - INFO - [retrieve_20251117_112746_642148] Searching vector database
2025-11-17 11:27:46 - LightMemory - INFO - [retrieve_20251117_112746_642148] Found 4 results
2025-11-17 11:27:46 - LightMemory - INFO - [retrieve_20251117_112746_642148] Formatted 4 results into output string
2025-11-17 11:27:46 - LightMemory - INFO - ========== END retrieve_20251117_112746_642148 ==========


Query: What are my travel preferences?
Retrieved memories:
2025-01-11T00:00:00.000 Sat User prefers cities with good public transportation.
2025-01-12T00:00:00.000 Sun User is planning a trip for spring 2025.
2025-01-10T00:00:00.000 Fri User loves visiting museums.
2025-01-10T00:00:00.000 Fri User loves trying local cuisine.

Total memories stored: 4


2025-11-17 11:27:46 - LightMemory - INFO - Initializing LightMemory with provided configuration
2025-11-17 11:27:46 - LightMemory - INFO - Initializing pre-compressor



FORKING: Create Branch A (Paris) and Branch B (Tokyo)
pre_compressor:llmlingua-2
pre_compressor:llmlingua_config={'model_name': 'microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank', 'device_map': 'cuda', 'use_llmlingua2': True} llmlingua2_config={'max_batch_size': 50, 'max_force_token': 100} compress_config={'instruction': '', 'rate': 0.8, 'target_token': -1}


2025-11-17 11:27:47 - LightMemory - INFO - Initializing topic segmenter
2025-11-17 11:27:47 - LightMemory - INFO - Initializing memory manager
2025-11-17 11:27:47 - LightMemory - INFO - Initializing text embedder
2025-11-17 11:27:47 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-11-17 11:27:47 - LightMemory - INFO - Initializing embedding retriever
2025-11-17 11:27:47 - LightMemory - INFO - LightMemory initialization completed successfully
2025-11-17 11:27:47 - LightMemory - INFO - Initializing LightMemory with provided configuration
2025-11-17 11:27:47 - LightMemory - INFO - Initializing pre-compressor


pre_compressor:llmlingua-2
pre_compressor:llmlingua_config={'model_name': 'microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank', 'device_map': 'cuda', 'use_llmlingua2': True} llmlingua2_config={'max_batch_size': 50, 'max_force_token': 100} compress_config={'instruction': '', 'rate': 0.8, 'target_token': -1}


2025-11-17 11:27:48 - LightMemory - INFO - Initializing topic segmenter
2025-11-17 11:27:48 - LightMemory - INFO - Initializing memory manager
2025-11-17 11:27:48 - LightMemory - INFO - Initializing text embedder
2025-11-17 11:27:48 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-11-17 11:27:48 - LightMemory - INFO - Initializing embedding retriever
2025-11-17 11:27:48 - LightMemory - INFO - LightMemory initialization completed successfully
2025-11-17 11:27:48 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection fork_test_branch_a
2025-11-17 11:27:48 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection fork_test_branch_b
2025-11-17 11:27:48 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection fork_test_branch_a
2025-11-17 11:27:48 - lightmem.factory.retriever.emb


Cloning pre-fork memories to branches...
Found 4 memories in main to clone
✓ Branch A initialized with 4 memories
✓ Branch B initialized with 4 memories

------------------------------------------------------------
Branch A: Testing hypothesis 'Paris'
------------------------------------------------------------


2025-11-17 11:27:49 - LightMemory - INFO - [add_memory_20251117_112748_924243] Metadata generation completed with 1 API calls
2025-11-17 11:27:49 - LightMemory - INFO - [add_memory_20251117_112748_924243] Extracted 1 memory entries
2025-11-17 11:27:49 - LightMemory - INFO - [add_memory_20251117_112748_924243] Created 0 MemoryEntry objects
2025-11-17 11:27:49 - LightMemory - INFO - ========== START offline_update_20251117_112749_818952 ==========
2025-11-17 11:27:49 - LightMemory - INFO - [offline_update_20251117_112749_818952] Received 0 memory entries
2025-11-17 11:27:49 - LightMemory - INFO - [offline_update_20251117_112749_818952] construct_update_queue_trigger=False, offline_update_trigger=False
2025-11-17 11:27:49 - LightMemory - INFO - [offline_update_20251117_112749_818952] Starting embedding and insertion to vector database
2025-11-17 11:27:49 - LightMemory - INFO - [offline_update_20251117_112749_818952] Successfully inserted 0 entries to vector database
2025-11-17 11:27:49 - 

✓ Added Paris hypothesis to Branch A

------------------------------------------------------------
Branch B: Testing hypothesis 'Tokyo'
------------------------------------------------------------


2025-11-17 11:27:50 - LightMemory - INFO - [add_memory_20251117_112749_821136] Metadata generation completed with 1 API calls
2025-11-17 11:27:50 - LightMemory - INFO - [add_memory_20251117_112749_821136] Extracted 1 memory entries
2025-11-17 11:27:50 - LightMemory - INFO - [add_memory_20251117_112749_821136] Created 0 MemoryEntry objects
2025-11-17 11:27:50 - LightMemory - INFO - ========== START offline_update_20251117_112750_813998 ==========
2025-11-17 11:27:50 - LightMemory - INFO - [offline_update_20251117_112750_813998] Received 0 memory entries
2025-11-17 11:27:50 - LightMemory - INFO - [offline_update_20251117_112750_813998] construct_update_queue_trigger=False, offline_update_trigger=False
2025-11-17 11:27:50 - LightMemory - INFO - [offline_update_20251117_112750_813998] Starting embedding and insertion to vector database
2025-11-17 11:27:50 - LightMemory - INFO - [offline_update_20251117_112750_813998] Successfully inserted 0 entries to vector database


✓ Added Tokyo hypothesis to Branch B


In [4]:
### Cell 3: Verify fork isolation and memory access

print("\n" + "=" * 60)
print("VERIFICATION: Test retrieval in each branch")
print("=" * 60)

# Query about travel preferences (should retrieve pre-fork memories)
query_prefork = "What are my travel preferences?"

print("\n[Main] Retrieve:", query_prefork)
main_retrieved = lightmem_main.retrieve(query_prefork, limit=3)
print(main_retrieved)
print(f"Main collection size: {len(lightmem_main.embedding_retriever.get_all())} memories")

print("\n" + "-" * 60)
print("[Branch A] Retrieve:", query_prefork)
branch_a_retrieved = lightmem_branch_a.retrieve(query_prefork, limit=3)
print(branch_a_retrieved)
print(f"Branch A collection size: {len(lightmem_branch_a.embedding_retriever.get_all())} memories")

print("\n" + "-" * 60)
print("[Branch B] Retrieve:", query_prefork)
branch_b_retrieved = lightmem_branch_b.retrieve(query_prefork, limit=3)
print(branch_b_retrieved)
print(f"Branch B collection size: {len(lightmem_branch_b.embedding_retriever.get_all())} memories")

# Query branch-specific memories (should only appear in respective branch)
print("\n" + "=" * 60)
print("ISOLATION TEST: Branch-specific memories")
print("=" * 60)

query_paris = "Paris destination"
print(f"\n[Branch A] Query: '{query_paris}'")
paris_in_a = lightmem_branch_a.retrieve(query_paris, limit=3)
print(paris_in_a)
print(f"  → Should mention Paris: {'Paris' in paris_in_a or 'museums' in paris_in_a}")

print(f"\n[Branch B] Query: '{query_paris}'")
paris_in_b = lightmem_branch_b.retrieve(query_paris, limit=3)
print(paris_in_b)
print(f"  → Should NOT mention Paris specifically: {'Paris' not in paris_in_b}")

query_tokyo = "Tokyo destination"
print(f"\n[Branch A] Query: '{query_tokyo}'")
tokyo_in_a = lightmem_branch_a.retrieve(query_tokyo, limit=3)
print(tokyo_in_a)
print(f"  → Should NOT mention Tokyo specifically: {'Tokyo' not in tokyo_in_a}")

print(f"\n[Branch B] Query: '{query_tokyo}'")
tokyo_in_b = lightmem_branch_b.retrieve(query_tokyo, limit=3)
print(tokyo_in_b)
print(f"  → Should mention Tokyo: {'Tokyo' in tokyo_in_b or 'food' in tokyo_in_b}")

print("\n" + "=" * 60)
print("✅ FORK TEST COMPLETE")
print("=" * 60)
print("Summary:")
print(f"  - Main has pre-fork memories only")
print(f"  - Branch A has pre-fork + Paris hypothesis")
print(f"  - Branch B has pre-fork + Tokyo hypothesis")
print(f"  - Branches are isolated (Paris ∉ Branch B, Tokyo ∉ Branch A)")


2025-11-17 11:27:50 - LightMemory - INFO - ========== START retrieve_20251117_112750_851614 ==========
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_851614] Query: What are my travel preferences?
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_851614] Parameters: limit=3, filters=None



VERIFICATION: Test retrieval in each branch

[Main] Retrieve: What are my travel preferences?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_851614] Searching vector database
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_851614] Found 3 results
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_851614] Formatted 3 results into output string
2025-11-17 11:27:50 - LightMemory - INFO - ========== END retrieve_20251117_112750_851614 ==========
2025-11-17 11:27:50 - LightMemory - INFO - ========== START retrieve_20251117_112750_865372 ==========
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_865372] Query: What are my travel preferences?
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_865372] Parameters: limit=3, filters=None


2025-01-11T00:00:00.000 Sat User prefers cities with good public transportation.
2025-01-12T00:00:00.000 Sun User is planning a trip for spring 2025.
2025-01-10T00:00:00.000 Fri User loves visiting museums.
Main collection size: 4 memories

------------------------------------------------------------
[Branch A] Retrieve: What are my travel preferences?


...

2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_914506] Searching vector database
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_914506] Found 3 results
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_914506] Formatted 3 results into output string
2025-11-17 11:27:50 - LightMemory - INFO - ========== END retrieve_20251117_112750_914506 ==========
2025-11-17 11:27:50 - LightMemory - INFO - ========== START retrieve_20251117_112750_927049 ==========
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_927049] Query: Tokyo destination
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_927049] Parameters: limit=3, filters=None


2025-01-11T00:00:00.000 Sat User prefers cities with good public transportation.
2025-01-12T00:00:00.000 Sun User is planning a trip for spring 2025.
2025-01-10T00:00:00.000 Fri User loves visiting museums.
  → Should NOT mention Tokyo specifically: True

[Branch B] Query: 'Tokyo destination'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_927049] Searching vector database
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_927049] Found 3 results
2025-11-17 11:27:50 - LightMemory - INFO - [retrieve_20251117_112750_927049] Formatted 3 results into output string
2025-11-17 11:27:50 - LightMemory - INFO - ========== END retrieve_20251117_112750_927049 ==========


2025-01-11T00:00:00.000 Sat User prefers cities with good public transportation.
2025-01-12T00:00:00.000 Sun User is planning a trip for spring 2025.
2025-01-10T00:00:00.000 Fri User loves visiting museums.
  → Should mention Tokyo: False

✅ FORK TEST COMPLETE
Summary:
  - Main has pre-fork memories only
  - Branch A has pre-fork + Paris hypothesis
  - Branch B has pre-fork + Tokyo hypothesis
  - Branches are isolated (Paris ∉ Branch B, Tokyo ∉ Branch A)
